In [322]:
import os
import numpy as np
import pandas as pd
import cv2
import skimage.io as sio
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
%matplotlib inline

In [326]:
targets = pd.read_csv('data/train_gt.csv', index_col='filename')
train_path = 'data/action-recognition-train/'
test_path = 'data/action-recognition-test/'
train_dir = 'data/train/'
test_dir = 'data/test/'

In [391]:
def save_frames(filename, old_dir, new_dir):
    video = cv2.VideoCapture(os.path.join(old_dir, filename))
    video.set(cv2.CAP_PROP_CONVERT_RGB, 1)
    duration = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_ids = np.round(np.linspace(0, duration, 10))[1: -1]
    os.makedirs(os.path.join(new_dir, filename[:4]), exist_ok=True)
    for i, frame_id in enumerate(frame_ids):
        video.set(cv2.CAP_PROP_POS_FRAMES, frame_id)
        _, frame = video.read()
        plt.imsave(os.path.join(new_dir, filename[:4], str(i)), cv2.cvtColor(frame, cv2.COLOR_RGBA2BGR))
        

def save_frames_train(filename):
    save_frames(filename, train_path, train_dir)

    
def save_frames_test(filename):
    save_frames(filename, test_path, test_dir)

In [392]:
from multiprocessing import Pool

with Pool(processes=4) as pool:
    [*tqdm_notebook(
        pool.imap(func=save_frames_train, iterable=os.listdir(train_path)), 
        total=len([*os.listdir(train_path)])
    )]
    [*tqdm_notebook(
        pool.imap(func=save_frames_test, iterable=os.listdir(test_path)), 
        total=len([*os.listdir(test_path)])
    )]